#### Implementação Bem Simples de um Classificador XGBoost

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

#### Carregar Dados de Treino

In [2]:
# Ler e carregar a base de dados
dados_treino = 'titanic_train.csv'
treino_df = pd.read_csv(dados_treino, header = 0)

# Examinar a base de dados
linhas, colunas = treino_df.shape # Extrair tamanho da base de dados
print('A base de dados de treino possui {} linhas e {} colunas.'.format(linhas, colunas)) # Apresentar informação ao usuário
treino_df.head(10) # Visualizar as 10 primeiras linhas da base de dados

A base de dados de treino possui 891 linhas e 12 colunas.


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


#### Carregar Dados de Teste

In [3]:
# Ler e carregar a base de dados
dados_teste = 'titanic_test.csv'
teste_df = pd.read_csv(dados_teste, header = 0)

# Examinar a base de dados
linhas, colunas = teste_df.shape # Extrair tamanho da base de dados
print('A base de dados de teste possui {} linhas e {} colunas.'.format(linhas, colunas)) # Apresentar informação ao usuário
treino_df.head(10) # Visualizar as 10 primeiras linhas da base de dados

A base de dados de teste possui 418 linhas e 11 colunas.


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


#### Preencher Valores Inexistentes e Tratar a Base de Dados de Treino e Teste

In [46]:
# Preencher valores inexistentes com a mediana
treino_df['Fare'] = treino_df['Fare'].fillna(treino_df['Fare'].median())
teste_df['Fare'] = teste_df['Fare'].fillna(teste_df['Fare'].median())

# Preencher valores inexistentes com 'S' de Southampton que é o mais comum
treino_df['Embarked'] = treino_df['Embarked'].fillna('S')
teste_df['Embarked'] = teste_df['Embarked'].fillna('S')

# Calcular número de membros da família
treino_df['Family'] = treino_df['SibSp'] + treino_df['Parch']
teste_df['Family'] = teste_df['SibSp'] + teste_df['Parch']

#### Converter Variáveis Categóricas em Variáveis Indicadores Dummy

In [47]:
# Definir função dummy
def add_dummy(df):
    df['Pclass'] = df['Pclass'].astype(np.str)
    temp = pd.get_dummies(df[['Sex','Embarked','Pclass']], drop_first = False)
    temp['PassengerId'] = df['PassengerId']
    return pd.merge(df, temp)

treino_df = add_dummy(treino_df) # Converter vairáveis categóricas dos dados de treino
teste_df = add_dummy(teste_df) # Converter vairáveis categóricas dos dados de teste

In [48]:
# Remover características desnecessárias
def get_feature_mat(df):
    temp = df.drop(columns = ['PassengerId','Name','Sex','SibSp','Parch','Ticket','Embarked','Age','Cabin','Pclass'])
    try:
        temp = temp.drop(columns = ['Survived'])
    except:
        pass
    # print (temp)
    return temp

X_treino = get_feature_mat(treino_df)
X_teste = get_feature_mat(teste_df)
y_treino = treino_df['Survived']

#### Treinar o Classificador XGBoost e Exportar suas Predições

In [62]:
# Definir estrutura e parâmetros de um classificador XGBoost
model = XGBClassifier(max_depth = 3, learning_rate = 0.1, n_estimators = 100, eval_metric = 'logloss') # Modelo definido
model.fit(X_treino, y_treino) # Treinar modelo com os dados de treino
predictions = model.predict(X_teste) # Predições do modelo já treinado para os dados de teste

# Exportar as predições do modelo na forma de um Dataframe
teste_df['Survived'] = predictions.astype(np.int)
teste_df[['PassengerId','Survived']].to_csv('submission.csv', index = False) # Salvar arquivo CSV no computador
print("Sua submissão foi salva com sucesso!")
print('Esse modelo alcançou uma pontuação pública na plataforma Kaggle igual a 0.78947 em uma escala de 0 (zero) a 1 (um).')

Sua submissão foi salva com sucesso!
Esse modelo alcançou uma pontuação pública na plataforma Kaggle igual a 0.78947 em uma escala de 0 (zero) a 1 (um).


In [61]:
# Definir estrutura e parâmetros de um classificador XGBoost
model = XGBClassifier(max_depth = 3, learning_rate = 0.1, n_estimators = 500, eval_metric = 'logloss') # Modelo definido
model.fit(X_treino, y_treino) # Treinar modelo com os dados de treino
predictions = model.predict(X_teste) # Predições do modelo já treinado para os dados de teste

# Exportar as predições do modelo na forma de um Dataframe
teste_df['Survived'] = predictions.astype(np.int)
teste_df[['PassengerId','Survived']].to_csv('submission.csv', index = False) # Salvar arquivo CSV no computador
print("Sua submissão foi salva com sucesso!")
print('Esse modelo alcançou uma pontuação pública na plataforma Kaggle igual a 0.77033 em uma escala de 0 (zero) a 1 (um).')

Sua submissão foi salva com sucesso!
Esse modelo alcançou uma pontuação pública na plataforma Kaggle igual a 0.77033 em uma escala de 0 (zero) a 1 (um).
